<a href="https://colab.research.google.com/github/chaene82/loki_jupyter/blob/master/Tensorflow_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupyter learning notebook for tensorflow 
Based on the book "Einfuehrung in Tenserflow"

```
# This is formatted as code
```



In [0]:
import tensorflow as tf